In [1]:
from pymongo import MongoClient
import pprint

import pandas as pd
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import requests
from bs4 import BeautifulSoup

import json
import time
import copy

import os

from time import sleep
from selenium import webdriver

import sys

In [3]:
driver = webdriver.Chrome()
driver.get('https://www.alltrails.com/trail/us/new-jersey/huylers-landing-trail')
sleep(3)

driver.execute_script("window.scrollTo(0, 10000)")

search = driver.find_element_by_css_selector("#reviews > div.styles-module__container___px-t2.xlate-none > button")
for _ in range(round(150/30)):
    search.click()
    sleep(1)

html = driver.page_source
time.sleep(2)

print(html)

In [141]:
# huyler_landing_url = 'https://www.alltrails.com/trail/us/new-jersey/huylers-landing-trail'
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
# r = requests.get(huyler_landing_url, headers=headers)

In [110]:
# client = MongoClient()

# db = client.alltrails

In [113]:
# pages = db.huyler_r

In [6]:
# pages.insert_one({'link': huyler_landing_url ,'html': html})

NameError: name 'pages' is not defined

In [7]:
soup = BeautifulSoup(html, 'html.parser')

In [8]:
# Trail title, description, difficulty, info
soup.find('h1', class_='xlate-none styles-module__name___1nEtW').text.rstrip()
soup.find('p', id="auto-overview").text.rstrip()
soup.find('span', class_="styles-module__diff___22Qtv styles-module__moderate___3w1it styles-module__selected___3fawg").text.rstrip()
soup.find('span', class_="styles-module__detailData___kQ-eK").text.rstrip()

'2.2 mi'

In [9]:
num_reviews = soup.find_all('div', itemtype="http://schema.org/Review")
len(num_reviews)
# reviews 
# # create a list of links to the cat images 
# # links_to_cats = []
# for i, review in enumerate(num_reviews): 
#     links_to_cats.append(cat.get_attribute('src'))

150

In [10]:
written_review = ""
ratings = []
dates = []
types = []
review_text = []
reviewer_links = []

for i, review in enumerate(soup.find_all('div', itemprop="review")):

    ratings.append(soup.find_all('span', class_="MuiRating-root default-module__rating___1k45X MuiRating-sizeLarge MuiRating-readOnly")[i]['aria-label'])
    dates.append(soup.find_all('span', class_="styles-module__dateTrailDetails___3qgZC xlate-none")[i].text.rstrip())
    types.append(soup.find_all('span', class_="styles-module__tag___2s-oD styles-module__activityTag___3-RdN")[i].text.rstrip())
    
    written_review = soup.find_all('div', class_="styles-module__container___3etfA")[i].find('p', itemprop="reviewBody")
    
    if written_review == None:
        review_text.append(None)

    else:
        review_text.append(soup.find_all('div', class_="styles-module__container___3etfA")[i].find('p', itemprop="reviewBody").text.rstrip())        
        
    reviewer_links.append('alltrails.com' + soup.find_all('div', class_="styles-module__container___3etfA")[1].find('a', class_="styles-module__link___2i6Za styles-module__recording___3jQX3 xlate-none")['href'])

In [11]:
df = pd.DataFrame({'rating': ratings, 'date':dates, 'desc': types, 'reviews': review_text, 'links': reviewer_links}).sort_values('rating')

In [41]:
five = df[df['rating'] == '5 Stars']['reviews'].count() #num of written reviews left by five-star reviewers
four = df[df['rating'] == '4 Stars']['reviews'].count() #num of written reviews left by four-star reviewers
num_five = df[df['rating']=='5 Stars']['rating'].count() #num of five-star reviewers
num_four = df[df['rating']=='4 Stars']['rating'].count() #num of four-star reviewers

table = pd.DataFrame({'Reviewers': [num_four, num_five], 'Written Reviews': [four, five]}, index = ['4 Stars', '5 Stars'])
tabletable.style.set_properties(**{'text-align': 'center'})


,Reviewers,Written Reviews
4 Stars,46,15
5 Stars,73,9


At first glance it is obvious that a much higher percentage of 4 star reviewers left written reviews than 5 star reviewers.